https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series


In [ ]:
baseurl = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_"
from pyspark.sql.types import *
import pandas as pd

In [ ]:
def saveFile(filetype):
    df = pd.read_csv(baseurl + filetype + "_global.csv")
    df = df.drop(columns=['Lat','Long'])
    df_unpivoted = df.melt(id_vars=['Province/State','Country/Region'], var_name='Date', value_name=filetype)
    df_unpivoted['Date'] = pd.to_datetime(df_unpivoted['Date'], format="%m/%d/%y")
    df_schema = StructType([ StructField("Province", StringType(), True),StructField("Country", StringType(), True),StructField("Date", DateType(), True),StructField(filetype, IntegerType(), True)])
    df_spark = spark.createDataFrame(df_unpivoted,df_schema)
    df_spark.createOrReplaceTempView("tmpView")
    df_spark = spark.sql("SELECT Province,Country,Date(Date)," + filetype + " FROM tmpView")
    df_spark.write.format('csv').option('header',True).mode('overwrite').save('/covid/csv/' + filetype)
    df_spark.write.format('parquet').mode('overwrite').saveAsTable(filetype,path='/covid/parquet/' + filetype)
    df_spark.sort(["Date",filetype], ascending=False).show(5)

In [ ]:
saveFile("recovered")
saveFile("deaths")
saveFile("confirmed")